In [53]:
import numpy as np
import pandas as pd
import math


In [74]:
class TreeNode:   
    def __init__(self, data,output):
        """
        HERE "data" REFERS TO THE FEATURE UPON WHICH THE 
        NODE WAS SPLIT WHEN FITTING THE TRAINING DATA
        
        IF THERE OCCURS A LEAF NODE THEN THE VALUE OF THE "data" 
        BECOMES "null" 
         
        THE CHILDREN OF A NODE ARE STORED IN A DICTIONARY WITH ITS KEY 
        BEING FEATURE UPON WHICH THE NODE HAS BEEN SPLITTED.
        
        CORRESPONDING VALUES STORE THE CHILD TREE NODE.
        """
        
        self.data = data
        
        self.children = {}
        """
        "output" REPRSENTS THE MAJORITY CLASS AT THAT CERTAIN POINT 
        OF DECISION
        """
        self.output = output
        """
        "index" PROVIDES AN UNIQUE INDEX TO EVERY DECISION TREE
        """
        self.index = -1
        
    def add_child(self,feature_value,obj):
        self.children[feature_value] = obj

In [75]:
class DecisionTreeClassifier:
    def __init__(self):
        """
        "root" CONSISTS OF THE ROOT NODE OF THE DECSION TREE
        
        """
        self.__root = None

    def __count_unique(self,Y):
        '''
        "d" IS A DICTIONARY THAT CONTAINS ALL THE UNIQUE CLASSES 
        IN Y AS KEYS AND THEIR CORRESPNDING FREQUENCY AS VALUE.
        THUS,FORMING A KEY-VALUE PAIR.
        
        '''
        d = {}
        for i in Y:
            if i not in d:
                d[i]=1
            else:
                d[i]+=1
        return d


    def __entropy(self,Y):
        '''
        THIS FUNCTION RETURNS THE ENTROPY
        
        '''
        freq_map = self.__count_unique(Y)
        entropy_ = 0
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            entropy_ += (-p)*math.log2(p)
        return entropy_

    def __gain_ratio(self,X,Y,selected_feature):
        '''
        THIS FUNCTION RETURNS THE GAIN RATIO
        
        '''
        '''
        "info_orig" STORES THE ENTROPY BEFORE SPLITTING
        
        '''
        info_orig = self.__entropy(Y) 
        '''
        "info_F" STORES THE ENTROPY AFTER SPLITTING
        
        '''
        
        info_f = 0  
        split_info = 0
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X)
        '''
        ADDING TARGET VALUES (Y) AS THE LAST COLUMN OF THE 
        DATAFRAME
        
        '''
        df[df.shape[1]] = Y
        initial_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            info_f += (current_size/initial_size)*self.__entropy(df1[df1.shape[1]-1])
            split_info += (-current_size/initial_size)*math.log2(current_size/initial_size)

        '''
        TO AVOID DIVIDE BY ZERO ERROR AS "split_info"
        CAN BE ZERO.        
        '''
        if split_info == 0 :
            return math.inf

        info_gain = info_orig - info_f
        gain_ratio = info_gain / split_info
        return gain_ratio

    def __gini_index(self,Y):
        '''
        THIS FUNCTION RETURNS THE "gini_index".
        
        '''
        freq_map = self.__count_unique(Y)
        gini_index_ = 1
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            gini_index_ -= p**2
        return gini_index_

    def __gini_gain(self,X,Y,selected_feature):
        '''
        THIS FUNCTION RETURNS THE "gini_gain_".
        
        '''
        '''
        "gini_orig" IS GAIN BEFORE SPLITTING
        
        '''
        gini_orig = self.__gini_index(Y) 
        '''
        "gini_split_f" IS GAIN AFTER SPLITTING
        
        '''
        gini_split_f = 0 
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X)
        '''
        ADDING TARGET VALUES (Y) AS THE LAST COLUMN OF THE 
        DATFRAME
        
        '''
        df[df.shape[1]] = Y
        initial_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            gini_split_f += (current_size/initial_size)*self.__gini_index(df1[df1.shape[1]-1])

        gini_gain_ = gini_orig - gini_split_f
        return gini_gain_


    def __decision_tree(self,X,Y,features,level,metric,classes):
        '''
         RETURNS THE ROOT OF THE DECISION TREE(WHICH CONSISTS OF THE  TREE NODES) BUILT AFTER FITTING 
         THE TRAINING DATA
         HERE NODES ARE PRINTED AS IN PREORDER TRAVERSL
         CLASSES REPRESENTS THE DIFFERENT CLASSES PRESENT IN THE CLASSIFICATION PROBLEM 
         METRIC CAN TAKE VALUE GAIN_RATIO OR GINI_INDEX
         LEVEL REPRESENTS DEPTH OF THE TREE
         WE SPLIT A NODE ON A PARTICULAR FEATURE ONLY ONCE (IN A GIVEN ROOT TO LEAF NODE PATH)

        '''
        
        '''
          IF NODE CONSISTS OF ONLY 1 CLASS
        '''
        if len(set(Y)) == 1:
            print("Level",level)
            output = None
            for i in classes:
                if i in Y:
                    output = i
                    print("Count of",i,"=",len(Y))
                else :
                    print("Count of",i,"=",0)
            if metric == "gain_ratio":
                print("Current Entropy is =  0.0")
            elif metric == "gini_index":
                print("Current Gini Index is =  0.0")

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)

        '''
          IF THERE ARE NO FEATURES AVAILABLE TO SPLIT THEN WE
          RETURN THE MAXIMUM COUNT
        '''
        if len(features) == 0:
            print("Level",level)
            freq_map = self.__count_unique(Y)
            output = None
            max_count = -math.inf
            for i in classes:
                if i not in freq_map:
                    print("Count of",i,"=",0)
                else :
                    if freq_map[i] > max_count :
                        output = i
                        max_count = freq_map[i]
                    print("Count of",i,"=",freq_map[i])

            if metric == "gain_ratio":
                print("Current Entropy  is =",self.__entropy(Y))
            elif metric == "gini_index":
                print("Current Gini Index is =",self.__gini_index(Y))            

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)

        
        '''
           FINDING THE BEST FEATURE THAT CAN BE SPLITTED
        '''
        max_gain = -math.inf
        final_feature = None
        for f in features :
            if metric == "gain_ratio":
                current_gain = self.__gain_ratio(X,Y,f)
            elif metric =="gini_index":
                current_gain = self.__gini_gain(X,Y,f)

            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f

        print("Level",level)
        freq_map = self.__count_unique(Y)
        output = None
        max_count = -math.inf

        for i in classes:
            if i not in freq_map:
                print("Count of",i,"=",0)
            else :
                if freq_map[i] > max_count :
                    output = i
                    max_count = freq_map[i]
                print("Count of",i,"=",freq_map[i])

        if metric == "gain_ratio" :        
            print("Current Entropy is =",self.__entropy(Y))
            print("Splitting on feature  X[",final_feature,"] with gain ratio ",max_gain,sep="")
            print()
        elif metric == "gini_index":
            print("Current Gini Index is =",self.__gini_index(Y))
            print("Splitting on feature  X[",final_feature,"] with gini gain ",max_gain,sep="")
            print()

        '''
          "unique_values "REPRESENTS THE UNIQUE VALUES OF THE FEATURE 
           SELECTED
        '''    
        unique_values = set(X[:,final_feature]) 
        df = pd.DataFrame(X)
        '''
         ADDING Y COLUMNN IN THE DATAFRAME IN THE END
        '''
        df[df.shape[1]] = Y

        current_node = TreeNode(final_feature,output)

        '''
          NOW REMOVING THE FEATURES THOSE ARE ALREADY SELECTED TO ENSURE 
          RE-SPLITTING
        '''
        index  = features.index(final_feature)
        features.remove(final_feature)
        for i in unique_values:
            '''
            CREATING A NEW DATAFRAME WITH THE VALUE OF SELECTED FEATURE BEING ONE
            '''
            df1 = df[df[final_feature] == i]
            '''
            SEGREGATING THE X AND Y VALUES AND RECURSIVELY CALLING THE SPLITS
            '''
            node = self.__decision_tree(df1.iloc[:,0:df1.shape[1]-1].values,df1.iloc[:,df1.shape[1]-1].values,features,level+1,metric,classes)
            current_node.add_child(i,node)

        '''
         ADDING THE REMOVED FEATURE
        '''
        features.insert(index,final_feature)

        return current_node
    
    def fit(self,X,Y,metric="gain_ratio"):
        '''
          THIS FUNCTION FITS THE TRAINING DATA
          METRIC CAN TAKE VALUE:"gini_index" AND
          "gini_ratio"
        '''
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        if metric != "gain_ratio" :
            if metric != "gini_index":
                '''
                JUST IN CASE NO VALUE IS PROVIDED TO METRIC THEN 
                THE DEFAULT VALUE WILL BE "gain_ratio"
                '''
                metric="gain_ratio" 
        self.__root = self.__decision_tree(X,Y,features,level,metric,classes)
        
    def __predict_for(self,data,node):
        '''
          PREDICTS THE CLASS FOR THE GIVEN DATA POINTS
        '''
    
        '''
          IF THE NODE HAS ONLY 1 FEATURE
        '''
        if len(node.children) == 0 :
            return node.output

        val = data[node.data] # represents the value of feature on which the split was made       
        if val not in node.children :
            return node.output
        
        '''
          RECUSIVELY CALLLING SPLITS
        '''
        return self.__predict_for(data,node.children[val])

    def predict(self,X):
        # This function returns Y predicted
        # X should be a 2-D np array
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.__predict_for(X[i],self.__root)
        return Y
    
    def score(self,X,Y):
        # returns the mean accuracy
        Y_pred = self.predict(X)
        count = 0
        for i in range(len(Y_pred)):
            if Y_pred[i] == Y[i]:
                count+=1
        return count/len(Y_pred)
    
    def export_tree_pdf(self,filename=None):
        # returns the tree as dot data
        # if filename is specified the function 
        # will save the pdf file in current directory which consists of the visual reresentation of the tree
        import pydotplus
        from collections import deque
        
        dot_data = '''digraph Tree {
node [shape=box] ;'''
        
        queue = deque()
        
        r = self.__root
        queue.append(r)
        count = 0
        if r.index == -1:
            r.index = count
        
        dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(count,r.data,r.output) 
        
        '''
          DOING LEVEL ORDER TRAVERSAL IN A TREE USING A QUEUE
        '''
        while len(queue) != 0 :
            node = queue.popleft()
            for i in node.children:
                count+=1
                if(node.children[i].index==-1):
                    node.children[i].index = count
                
                '''
                CREATING CHILD
                '''
                dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(node.children[i].index,node.children[i].data,node.children[i].output) 
                '''CONNECTING PARENT WITH CHILD NODES
                   
                '''
                dot_data = dot_data + "\n{} -> {} [ headlabel=\"Feature value = {}\"]; ".format(node.index,node.children[i].index,i)
                '''
                ADDING CHILD NODE TO QUEUE
                '''
                queue.append(node.children[i])
        
        dot_data = dot_data + "\n}"

        if filename != None:    
            graph = pydotplus.graph_from_dot_data(dot_data)
            graph.write_pdf(filename)    
        
        return dot_data

In [76]:
from sklearn import datasets

from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split

In [77]:
'''
LOADING THE IRIS DATASET USING SKLEARN DATASETS AND MAKING IT READY 
FOR TRAINING AND TESTING
'''

iris = datasets.load_iris()

x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state = 1)

In [78]:

clf2 = DecisionTreeClassifier()
clf2.fit(x_train,y_train,metric='gini_index')
Y_pred2 = clf2.predict(x_test)
print("Predictions : ",Y_pred2)
print()
our_score = clf2.score(x_train,y_train)
'''
CALCULATING THE TRAINING SCORE
'''
print("Score :",our_score) 
print()



Level 0
Count of 0 = 37
Count of 1 = 34
Count of 2 = 41
Current Gini Index is = 0.6647002551020409
Splitting on feature  X[2] with gini gain 0.5971407312925171

Level 1
Count of 0 = 11
Count of 1 = 0
Count of 2 = 0
Current Gini Index is =  0.0
Reached leaf Node

Level 1
Count of 0 = 3
Count of 1 = 0
Count of 2 = 0
Current Gini Index is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 2
Count of 2 = 0
Current Gini Index is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 2
Count of 2 = 0
Current Gini Index is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 3
Current Gini Index is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 2
Current Gini Index is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 3
Current Gini Index is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 5
Current Gini Index is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count o

In [79]:
'''
GENERATES THE DECSION TREE AND STORES IT IN A PDF
'''
clf2.export_tree_pdf(filename="Decision_Tree_Assignment.pdf")


'digraph Tree {\nnode [shape=box] ;\n0 [label="Feature to split upon : X[2]\\nOutput at this node : 2" ];\n1 [label="Feature to split upon : X[None]\\nOutput at this node : 0" ];\n0 -> 1 [ headlabel="Feature value = 1.4"]; \n2 [label="Feature to split upon : X[None]\\nOutput at this node : 0" ];\n0 -> 2 [ headlabel="Feature value = 1.7"]; \n3 [label="Feature to split upon : X[None]\\nOutput at this node : 1" ];\n0 -> 3 [ headlabel="Feature value = 3.9"]; \n4 [label="Feature to split upon : X[None]\\nOutput at this node : 1" ];\n0 -> 4 [ headlabel="Feature value = 4.6"]; \n5 [label="Feature to split upon : X[None]\\nOutput at this node : 2" ];\n0 -> 5 [ headlabel="Feature value = 5.5"]; \n6 [label="Feature to split upon : X[None]\\nOutput at this node : 2" ];\n0 -> 6 [ headlabel="Feature value = 5.0"]; \n7 [label="Feature to split upon : X[None]\\nOutput at this node : 2" ];\n0 -> 7 [ headlabel="Feature value = 5.8"]; \n8 [label="Feature to split upon : X[None]\\nOutput at this node : 2